# MC-Test Streamlit App Cloud Deployment Guide

Dieses Notebook beschreibt den minimalen und vereinfachten Deploy der *Multiple-Choice Test Streamlit App* (`mc_test_app/mc_test_app.py`) auf die **Streamlit Community Cloud**. Komplexe Alternativ-Strategien (temporäre Branches, Skript, Makefile) wurden entfernt zugunsten eines einzigen klaren Befehls.

## 1. Überblick & Architektur

Die App schreibt Antworten der Nutzer in eine CSV (`mc_test_answers.csv`). Auf der Community Cloud ist das Dateisystem persistierend pro Deployment, kann aber bei Redeploy/History-Reset zurückgesetzt werden. Für produktive Persistenz später externe Dienste (Sheets, DB, S3) nutzen.

**Komponenten:**
- `mc_test_app.py` – Haupt-App
- `mc_test_answers.csv` – Log-Datei (append-only)
- `mc_test_app/requirements.txt` – Minimal-Dependencies für Cloud
- Root `requirements.txt` – Voller Kurs-Stack (lokal)
- `.streamlit/secrets.toml` (optional) oder Secrets UI

> Empfehlung: Für die Cloud nur `mc_test_app/requirements.txt` verwenden.

**Architektur (vereinfacht)**
```
Browser → Streamlit Cloud → mc_test_app.py → CSV
```

## 2. Minimal notwendige Dateien
```
mc_test_app/mc_test_app.py
mc_test_app/requirements.txt
mc_test_app/questions.json
mc_test_app/tests/
(optional) mc_test_app/mc_test_answers.csv
```
> Die CSV wird automatisch erzeugt, falls nicht vorhanden.

## 3. Dependencies (minimal)
```
streamlit>=1.32
pandas>=2.2
numpy>=1.26
```

## 4. Admin-Key / Secrets
In der Cloud UI unter Secrets setzen:
```toml
MC_TEST_ADMIN_KEY = "Admin"
```

## 5. Lokaler Schnelltest
```bash
streamlit run mc_test_app/mc_test_app.py
```

## 6. Deployment (Einzeiler)
Ein einziger Befehl pusht ausschließlich den Unterordner `mc_test_app/` auf den Branch `main` des Remotes (hier: `github`).
```bash
git subtree push --prefix mc_test_app github main
```
Voraussetzungen:
- Änderungen im Unterordner committed
- Remote heißt `github` (sonst `origin` verwenden)

Fehler: "non-fast-forward" oder Divergenz → zuerst lokalen `main` aktualisieren:
```bash
git pull --ff-only github main
git subtree push --prefix mc_test_app github main
```

## 7. CI (Tests)
Zwei Jobs (Minimal + Full) in `.github/workflows/ci.yml` stellen Funktionalität sicher. Der Deploy-Einzeiler kann unabhängig davon lokal gelaufen werden.

## 8. Streamlit Cloud Setup
1. New App
2. Repo auswählen
3. Branch: `main`
4. App Path: `mc_test_app/mc_test_app.py`
5. Requirements file: `mc_test_app/requirements.txt`
6. Deploy

## 9. Monitoring
Logs im Cloud Dashboard prüfen (Build, Laufzeitfehler).

## 10. CSV Persistenz – Alternativen
Für dauerhafte/skalierte Nutzung: Sheets, Postgres (Supabase), S3.

## 11. Erweiterungen (Ideen)
- Mehrsprachigkeit
- Erweiterte Leaderboards
- Externe DB
- Analytics (Datenschutz beachten)

## 12. Checkliste
- [ ] Lokaler Start ok
- [ ] Tests grün
- [ ] Secrets gesetzt
- [ ] App-Pfad korrekt
- [ ] Keine sensiblen Daten im Repo

## 13. Zusammenfassung
Einzeiler-Deploy: `git subtree push --prefix mc_test_app github main`. Keine Force-Pushes oder Zusatzskripte nötig. Minimaler, reproduzierbarer Workflow.